In [2]:
import pandas as pd

In [3]:
df1 = pd.read_csv('country_vaccinations.csv')
df2 = pd.read_csv('country_vaccinations_by_manufacturer.csv')

In [4]:
df1.head()

,country,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million,vaccines,source_name,source_website
0,Afghanistan,AFG,2021-02-22,0.0,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/
1,Afghanistan,AFG,2021-02-23,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,34.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/
2,Afghanistan,AFG,2021-02-24,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,34.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/
3,Afghanistan,AFG,2021-02-25,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,34.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/
4,Afghanistan,AFG,2021-02-26,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,34.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/


In [5]:
df1['formatted_date'] = pd.to_datetime(df1['date'])

In [6]:
df1[df1['iso_code']=='AFG']['formatted_date'].max()

Timestamp('2021-09-30 00:00:00')

In [7]:
df1[df1['iso_code']=='AFG']['formatted_date'].count()

221

In [8]:
df1[['iso_code','total_vaccinations']]

,iso_code,total_vaccinations
0,AFG,0.0
1,AFG,NaN
2,AFG,NaN
3,AFG,NaN
4,AFG,NaN
...,...,...
49838,ZWE,5472485.0
49839,ZWE,5499530.0
49840,ZWE,5527583.0
49841,ZWE,NaN


In [9]:
total_vaccinations=df1.groupby(['iso_code'],group_keys = False).agg({'formatted_date': 'max','total_vaccinations':'max'}).reset_index()

In [10]:
df1.isna().sum()

country                                    0
iso_code                                   0
date                                       0
total_vaccinations                     22606
people_vaccinated                      24035
people_fully_vaccinated                27031
daily_vaccinations_raw                 27477
daily_vaccinations                       247
total_vaccinations_per_hundred         22606
people_vaccinated_per_hundred          24035
people_fully_vaccinated_per_hundred    27031
daily_vaccinations_per_million           247
vaccines                                   0
source_name                                0
source_website                             0
formatted_date                             0
dtype: int64

In [11]:
df1.shape

(49843, 16)

In [12]:
df2.isna().sum()

location              0
date                  0
vaccine               0
total_vaccinations    0
dtype: int64

In [13]:
countries = df1['country'].unique().tolist()

In [14]:
print(len(countries))

222


In [15]:
df2.head()

,location,date,vaccine,total_vaccinations
0,Austria,2021-01-08,Johnson&Johnson,0
1,Austria,2021-01-08,Moderna,0
2,Austria,2021-01-08,Oxford/AstraZeneca,0
3,Austria,2021-01-08,Pfizer/BioNTech,31131
4,Austria,2021-01-15,Johnson&Johnson,0


In [16]:
df2.location.unique()

array(['Austria', 'Belgium', 'Bulgaria', 'Chile', 'Croatia', 'Cyprus',
       'Czechia', 'Denmark', 'Ecuador', 'Estonia', 'Finland', 'France',
       'Germany', 'Hong Kong', 'Hungary', 'Iceland', 'Ireland', 'Italy',
       'Japan', 'Latvia', 'Liechtenstein', 'Lithuania', 'Luxembourg',
       'Malta', 'Netherlands', 'Norway', 'Poland', 'Portugal', 'Romania',
       'Slovakia', 'Slovenia', 'South Korea', 'Spain', 'Sweden',
       'Switzerland', 'United States', 'Uruguay', 'European Union'],
      dtype=object)

In [17]:
df2[df2['location'] == 'Austria'].groupby('vaccine').agg({'total_vaccinations':'max','date':'max'}).head(10)

,total_vaccinations,date
vaccine,,
Johnson&Johnson,314254,2021-10-08
Moderna,1025849,2021-10-08
Oxford/AstraZeneca,1597439,2021-10-08
Pfizer/BioNTech,7566430,2021-10-08


In [18]:
df1[df1['country']=='Austria'][['total_vaccinations','date']].max()

total_vaccinations    10948111.0
date                  2021-10-09
dtype: object

In [19]:
total = 314254 + 1025849 + 1597439 +7566430

In [20]:
print(total)

10503972


In [29]:
def get_vaccine_distribution_by_manufacturer(data,k):
    data = data.groupby(['location','vaccine'],group_keys = False).agg({'total_vaccinations':'max'}).reset_index()
    topk_countries = data.groupby(['location'],group_keys = False).agg({'total_vaccinations':'sum'}).reset_index().sort_values('total_vaccinations',ascending = False)['location'].iloc[0:k].tolist()
    data = data[data['location'].isin(topk_countries)]
    return data
    

In [30]:
data = get_vaccine_distribution_by_manufacturer(df2,10)

In [31]:
data.head()

,location,vaccine,total_vaccinations
12,Chile,CanSino,568719
13,Chile,Oxford/AstraZeneca,514740
14,Chile,Pfizer/BioNTech,7323409
15,Chile,Sinovac,21080458
40,European Union,Johnson&Johnson,14937080


In [32]:
data = get_vaccine_distribution_by_manufacturer(df2,10).sort_values('total_vaccinations',ascending = False)
data['location'].iloc[0:10].tolist()


['European Union',
 'United States',
 'United States',
 'Japan',
 'Germany',
 'France',
 'European Union',
 'Italy',
 'European Union',
 'Spain']

In [33]:
k =10
get_vaccine_distribution_by_manufacturer(df2,k)

,location,vaccine,total_vaccinations
12,Chile,CanSino,568719
13,Chile,Oxford/AstraZeneca,514740
14,Chile,Pfizer/BioNTech,7323409
15,Chile,Sinovac,21080458
40,European Union,Johnson&Johnson,14937080
41,European Union,Moderna,57676732
42,European Union,Oxford/AstraZeneca,67150901
43,European Union,Pfizer/BioNTech,403612939
44,European Union,Sinopharm/Beijing,2108368
45,European Union,Sputnik V,1843001


In [34]:
len(df2['location'].unique())

38

In [35]:
df2.isna().sum()

location              0
date                  0
vaccine               0
total_vaccinations    0
dtype: int64

In [36]:
df1.columns

Index(['country', 'iso_code', 'date', 'total_vaccinations',
       'people_vaccinated', 'people_fully_vaccinated',
       'daily_vaccinations_raw', 'daily_vaccinations',
       'total_vaccinations_per_hundred', 'people_vaccinated_per_hundred',
       'people_fully_vaccinated_per_hundred', 'daily_vaccinations_per_million',
       'vaccines', 'source_name', 'source_website', 'formatted_date'],
      dtype='object')

In [37]:
df1.head()

,country,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million,vaccines,source_name,source_website,formatted_date
0,Afghanistan,AFG,2021-02-22,0.0,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/,2021-02-22
1,Afghanistan,AFG,2021-02-23,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,34.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/,2021-02-23
2,Afghanistan,AFG,2021-02-24,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,34.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/,2021-02-24
3,Afghanistan,AFG,2021-02-25,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,34.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/,2021-02-25
4,Afghanistan,AFG,2021-02-26,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,34.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/,2021-02-26


In [38]:
df1.groupby(['iso_code'],group_keys= False).agg({'people_fully_vaccinated_per_hundred':'max'}).reset_index()

,iso_code,people_fully_vaccinated_per_hundred
0,ABW,70.52
1,AFG,1.08
2,AGO,2.97
3,AIA,60.54
4,ALB,28.85
...,...,...
217,WSM,28.79
218,YEM,0.16
219,ZAF,16.13
220,ZMB,2.42


In [39]:
l1=df1['vaccines'].unique().tolist()
vaccines = set()
print(l1)
for ele in l1:
    print(set(ele.split(',')))
    vaccines=vaccines.union(set(ele.split(',')))

['Johnson&Johnson, Oxford/AstraZeneca, Pfizer/BioNTech, Sinopharm/Beijing', 'Oxford/AstraZeneca, Pfizer/BioNTech, Sinovac, Sputnik V', 'Oxford/AstraZeneca, Sinopharm/Beijing, Sputnik V', 'Oxford/AstraZeneca, Pfizer/BioNTech', 'Oxford/AstraZeneca', 'Oxford/AstraZeneca, Pfizer/BioNTech, Sputnik V', 'CanSino, Moderna, Oxford/AstraZeneca, Pfizer/BioNTech, Sinopharm/Beijing, Sputnik V', 'Oxford/AstraZeneca, Sinopharm/Beijing, Sinovac, Sputnik V', 'Pfizer/BioNTech', 'Moderna, Oxford/AstraZeneca, Pfizer/BioNTech', 'Johnson&Johnson, Moderna, Oxford/AstraZeneca, Pfizer/BioNTech', 'Johnson&Johnson, Oxford/AstraZeneca, Pfizer/BioNTech', 'Oxford/AstraZeneca, Pfizer/BioNTech, Sinopharm/Beijing, Sputnik V', 'Moderna, Oxford/AstraZeneca, Pfizer/BioNTech, Sinopharm/Beijing', 'Oxford/AstraZeneca, Pfizer/BioNTech, Sinopharm/Beijing', 'Sinopharm/Beijing, Sputnik V', 'Oxford/AstraZeneca, Sinopharm/Beijing', 'Oxford/AstraZeneca, Sinovac', 'Johnson&Johnson, Oxford/AstraZeneca, Pfizer/BioNTech, Sinopharm/Bei

In [40]:
print(vaccines)

{'Covaxin', 'Moderna', ' Oxford/AstraZeneca', 'Oxford/AstraZeneca', ' Sputnik V', ' Sinopharm/Beijing', ' Sinovac', 'EpiVacCorona', 'Medigen', 'Abdala', ' Johnson&Johnson', 'Sputnik V', 'Pfizer/BioNTech', 'CanSino', 'Sinopharm/Beijing', ' Covaxin', ' Sputnik Light', ' ZF2001', ' Pfizer/BioNTech', 'QazVac', ' Soberana02', 'Johnson&Johnson', ' Sinopharm/Wuhan', ' Moderna', 'COVIran Barekat'}


In [41]:
df1.head()

,country,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million,vaccines,source_name,source_website,formatted_date
0,Afghanistan,AFG,2021-02-22,0.0,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/,2021-02-22
1,Afghanistan,AFG,2021-02-23,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,34.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/,2021-02-23
2,Afghanistan,AFG,2021-02-24,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,34.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/,2021-02-24
3,Afghanistan,AFG,2021-02-25,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,34.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/,2021-02-25
4,Afghanistan,AFG,2021-02-26,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,34.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/,2021-02-26


In [45]:
total_vaccinations=df1.groupby(['iso_code'],group_keys = False).agg({'formatted_date': 'max','total_vaccinations':'max','people_fully_vaccinated':'max'}).reset_index()

In [47]:
total_vaccinations['total_vaccinations'].sum()

6577920579.0

In [48]:
total_vaccinations['people_fully_vaccinated'].sum()

2797512520.0

In [53]:
df2.head()

,location,date,vaccine,total_vaccinations
0,Austria,2021-01-08,Johnson&Johnson,0
1,Austria,2021-01-08,Moderna,0
2,Austria,2021-01-08,Oxford/AstraZeneca,0
3,Austria,2021-01-08,Pfizer/BioNTech,31131
4,Austria,2021-01-15,Johnson&Johnson,0


In [54]:
popular_vaccines = df2.groupby(['location','vaccine'],group_keys = False).agg({'date': 'max','total_vaccinations':'max'}).reset_index()

In [57]:
popular_vaccines.groupby(['vaccine'],group_keys = False).agg({'total_vaccinations':'sum'}).reset_index()

,vaccine,total_vaccinations
0,CanSino,940961
1,Johnson&Johnson,46570659
2,Moderna,291024350
3,Oxford/AstraZeneca,159144763
4,Pfizer/BioNTech,1195125273
5,Sinopharm/Beijing,4216736
6,Sinovac,39724383
7,Sputnik V,3686002
